In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv \
    -O /tmp/training_cleaned.csv

--2020-06-10 08:15:20--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238942690 (228M) [application/octet-stream]
Saving to: ‘/tmp/training_cleaned.csv’

/tmp/training_clean 100%[===================>] 227.87M   123MB/s    in 1.8s    

2020-06-10 08:15:22 (123 MB/s) - ‘/tmp/training_cleaned.csv’ saved [238942690/238942690]



In [153]:
import numpy  as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import csv
import random

In [269]:
sentence,label=[],[]
total_list=[]
with open('/tmp/training_cleaned.csv','r') as clean:
    csv_tweets=csv.reader(clean)
    #random.shuffle(csv_tweets)
    for lines in csv_tweets:
        tweets_list=[]
        tweets_list.append(int(lines[0]))
        tweets_list.append(lines[5])
        total_list.append(tweets_list)

In [273]:
random.shuffle(tweets_list)
sentence,label=[],[]
for lines in total_list:
    label.append(lines[0])
    sentence.append(lines[1])

In [274]:
#sentence=sentence[500000:1000000]
#label=label[500000:1000000]

In [275]:
len(tweets_list)

2

In [315]:
total_list[1544556]

[4, 'ty, Monday is over ']

In [276]:
len(sentence)

500000

In [277]:
from sklearn.model_selection import train_test_split

In [278]:
## Data Splitting
train_sen=sentence[:300000]
train_label=label[:300000]
test_sen=sentence[300000:]
test_label=label[300000:]

In [279]:
len(train_sen)

300000

In [280]:
len(test_sen)

200000

In [281]:
train_sen=np.asarray(train_sen)
test_sen=np.asarray(test_sen)
train_label=np.asarray(train_label)
test_label=np.asarray(test_label)

In [282]:
train_sen.shape

(300000,)

In [283]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [284]:
token=Tokenizer()
token.fit_on_texts(train_sen)
word_index=token.word_index
vocab_size=len(word_index) +1

In [285]:
vocab_size

190182

In [286]:
train_seq=token.texts_to_sequences(train_sen)
test_seq=token.texts_to_sequences(test_sen)

In [287]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [288]:
train_pad=pad_sequences(train_seq,maxlen=20,padding='post',truncating='post')
test_pad_seq=pad_sequences(test_seq,maxlen=20,padding='post',truncating='post')

In [289]:
train_pad[135454]

array([118316,     11,    491,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0], dtype=int32)

In [290]:
train_sen[135454]

'@stephyyDUH , you suck '

### Lets load the weights of embedding from external source

In [291]:
### Get the Embedding weights from glove data set from Standford.
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt

--2020-06-10 11:11:18--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347116733 (331M) [text/plain]
Saving to: ‘/tmp/glove.6B.100d.txt’

/tmp/glove.6B.100d. 100%[===================>] 331.04M   148MB/s    in 2.2s    

2020-06-10 11:11:21 (148 MB/s) - ‘/tmp/glove.6B.100d.txt’ saved [347116733/347116733]



In [292]:
word_embedding={}
with open('/tmp/glove.6B.100d.txt') as glove:
    glove_text=glove.readlines()
    for lines in glove_text:
        line_list=lines.split()
        word_embedding[line_list[0]]=np.array(line_list[1:],dtype=np.float32)
        

In [293]:
word_embedding['forget'].shape

(100,)

In [294]:

vector=np.zeros((vocab_size,100),dtype=np.float32)
for word,index in  word_index.items():
    present=word_embedding.get(word)
    if present is not None:
        vector[index]=word_embedding[word]

In [295]:
vector.shape

(190182, 100)

In [296]:
word_index["i'm"]

19

In [297]:
index

190181

In [298]:
vocab_size

190182

In [299]:
reverse_word=dict([(w,i) for i,w in word_index.items()])

In [300]:
reverse_word[50002]

'msundeniablyfly'

In [301]:
vector[50000]

array([-2.3540e-01, -1.8496e-02, -1.4567e-01,  3.1103e-01, -4.6005e-01,
       -2.2487e-01,  5.0459e-01, -2.2457e-02,  1.3153e-02, -6.2834e-01,
        3.4538e-01,  7.5317e-02, -2.2186e-01,  9.0258e-02,  3.6487e-01,
        2.2868e-01, -2.2905e-01,  1.5696e-01,  2.6069e-01,  3.2342e-01,
        2.2956e-01,  5.1572e-01, -4.6915e-01, -1.7627e-01, -4.4410e-01,
        4.7118e-02,  1.6947e-01,  9.0651e-02,  2.6684e-01,  4.0472e-03,
       -5.4793e-02, -4.9479e-01,  1.6348e-01, -3.2987e-02,  6.2340e-01,
       -2.3301e-01,  2.3603e-01, -2.2212e-01,  5.5044e-01,  2.1335e-01,
        1.6117e-01,  2.8339e-01, -2.6120e-01, -2.2680e-01,  3.6699e-01,
       -2.5555e-02, -1.4005e-01,  1.7506e-01, -3.0135e-01,  3.4864e-01,
       -5.4640e-01,  7.8034e-01, -3.8328e-01, -2.0968e-01,  3.5590e-01,
        2.0367e-01,  4.2785e-01,  2.7563e-01, -8.5572e-01, -5.9034e-02,
        3.1618e-02, -3.7324e-01, -3.3288e-01,  7.2047e-02, -1.5621e-01,
       -9.1008e-02, -5.8744e-01, -6.7096e-01, -6.6506e-02,  1.97

## Model creation

In [302]:
from tensorflow.keras.layers import Embedding,GlobalAveragePooling1D,Dense,LSTM,Bidirectional,Dropout

In [303]:
model=keras.Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=100,input_length=20,weights=[vector],trainable=False))
model.add(LSTM(64))
model.add(Dropout(rate=0.3))
model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 20, 100)           19018200  
_________________________________________________________________
lstm_17 (LSTM)               (None, 64)                42240     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_28 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 129       
Total params: 19,068,889
Trainable params: 50,689
Non-trainable params: 19,018,200
_________________________________________________________________


In [304]:
model.fit(train_pad,train_label,epochs=5,steps_per_epoch=1000,batch_size=50,validation_data=(test_pad_seq,test_label))

Train on 300000 samples, validate on 200000 samples
Epoch 1/5
 50000/300000 [====>.........................] - ETA: 4:03 - loss: 3.0641e-08 - accuracy: 1.0000 - val_loss: 76.6929 - val_accuracy: 0.0000e+00

In [305]:
test_label.shape

(200000,)

In [306]:
test_label[:500]

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

In [307]:
sentence[52356]

"@JaiBrandon he's ok. nothing special. too much airbrushing "

In [308]:
test_label.max()

4

In [309]:
train_label.max()

0